# Lookup Table - Create Data

In [1]:
import scooby
import numpy as np

## Forward model: 3 model parameters (`p`, `q`, `r`); 5 measurements

In [2]:
def forward(p, q, r):
    """Simple set of five linear equations, simulating 5 measurements."""
    
    data1 = 3*p + 4*q + 5*r
    data2 = 2*p + 5*q + 2*r
    data3 = 4*p + 9*q + 0*r
    data4 = 0*p + 9*q + 4*r
    data5 = 4*p + 0*q + 6*r
       
    return np.stack([data1, data2, data3, data4, data5], axis=-1)

## Generate Lookup Table

Assumption:

- p: #60; 1 < p < 5
- q: #50; 0 < q < 2
- r: #40; 10 < r < 70

In [3]:
pp = np.linspace(1, 5, 60)
qq = np.linspace(0, 2, 50)
rr = np.linspace(10, 70, 40)

# One big table
lookuptable = forward(pp[:, None, None], qq[None, :, None], rr[None, None, :])

with open('lookuptable.npz', 'bw') as f:
    np.savez(f, lookuptable=lookuptable, pp=pp, qq=qq, rr=rr)

print(f"Shape: (p, q, r, ndat): {lookuptable.shape}")

# Split table by data; sort and store

def sort_table(table):
    i = np.argsort(table, axis=None).astype(np.int32)
    ts = table.ravel()[i]
    return i, ts

# Sort the arrays
for i in range(lookuptable.shape[-1]):
    ilt, lookuptable_sorted = sort_table(lookuptable[..., i])

    with open(f"lookuptable-sorted{i}.npz", 'bw') as f:
        np.savez(f, lookuptable=lookuptable_sorted, indices=ilt, pp=pp, qq=qq, rr=rr)

Shape: (p, q, r, ndat): (60, 50, 40, 5)


## Create some synthetic data

In [4]:
# "True" model parameters
p1, q1, r1 = 2.7, 1.1, 50.0
p2, q2, r2 = 5, 0, 70
p3, q3, r3 = 3.5, 1, 45



# "Measured" data
data1 = np.array(forward(p1, q1, r1), dtype=np.float32)  # Regular
data2 = np.array(forward(p2, q2, r2), dtype=np.float32)  # Limits
data3 = np.array(forward(p3, q3, r3), dtype=np.float32)  # With nose
data3 += [5, -2, -1, 3, 7]                          # ^ ^ ^

print(f"Synthetic data1: {data1}")
print(f"Synthetic data2: {data2}")
print(f"Synthetic data3: {data3}")

with open('lut-data.npz', 'bw') as f:
    np.savez(f, data1=data1, data2=data2, data3=data3,
             true1=(p1, q1, r1), true2=(p2, q2, r2), true3=(p3, q3, r3))

Synthetic data1: [262.5 110.9  20.7 209.9 310.8]
Synthetic data2: [365. 150.  20. 280. 440.]
Synthetic data3: [244.5 100.   22.  192.  291. ]


In [5]:
scooby.Report()

--------------------------------------------------------------------------------
  Date: Fri Apr 21 20:00:37 2023 CEST

                OS : Linux
            CPU(s) : 4
           Machine : x86_64
      Architecture : 64bit
               RAM : 15.5 GiB
       Environment : Jupyter
       File system : ext4

  Python 3.10.9 | packaged by conda-forge | (main, Feb  2 2023, 20:20:04) [GCC
  11.3.0]

             numpy : 1.23.5
             scipy : 1.10.0
           IPython : 8.9.0
        matplotlib : 3.6.3
            scooby : 0.7.1

  Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804
  for Intel(R) 64 architecture applications
--------------------------------------------------------------------------------